In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import numpy as np
a = np.array([1])
np.expand_dims(a,1).shape

(1, 1)

In [ ]:
import pandas as pd
data_df = pd.read_pickle('/content/drive/My Drive/GNN_df')
display(data_df)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
1,2019-10-01 00:00:00 UTC,0,0,0,appliances.environment.water_heater,aqua,33.20,0,9333dfbd-b87a-4708-9857-6336556b0fcc
3,2019-10-01 00:00:01 UTC,0,1,1,computers.notebook,lenovo,251.74,1,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,0,2,2,electronics.smartphone,apple,1081.98,2,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
5,2019-10-01 00:00:05 UTC,0,3,3,computers.desktop,pulser,908.62,3,0d0d91c2-c9c2-4e81-90a5-86594dec0db9
8,2019-10-01 00:00:10 UTC,0,4,4,apparel.shoes.keds,baden,102.71,4,ac1cd4e5-a3ce-4224-a2d7-ff660a105880
...,...,...,...,...,...,...,...,...,...
42444659,2019-10-31 23:25:33 UTC,0,37,2,electronics.smartphone,samsung,717.39,1770,c27e18bb-4f9e-46bb-9a00-bb122cd32c96
42444931,2019-10-31 23:28:15 UTC,0,331,2,electronics.smartphone,apple,1013.86,1743,bb7cae5a-ce49-4226-b268-b77ea51e8614
42445113,2019-10-31 23:29:51 UTC,0,331,2,electronics.smartphone,apple,1013.86,1743,bb7cae5a-ce49-4226-b268-b77ea51e8614
42445150,2019-10-31 23:30:11 UTC,0,950,2,electronics.smartphone,apple,1090.89,1743,bb7cae5a-ce49-4226-b268-b77ea51e8614


In [ ]:
#0-view 1-purchase 2-cart
total_products = data_df['product_id'].max()+1
total_users = data_df['user_id'].max()+1

# print(data_df['event_type'].unique())

# print('Product ids count = ', data_df['product_id'].max()+1)
# print('User ids count = ', data_df['user_id'].max()+1)

### **Building co-view matix**

In [ ]:
import numpy as np

#co-view matrix
session_column_name = 'user_session'
product_id_column_name = 'product_id'
event_type_column_name = 'event_type'
user_sessions = data_df[session_column_name].unique()
co_view_dict = {session: [] for session in user_sessions}
#iterate dataframe
for index, row in data_df.iterrows():
  if(row[event_type_column_name] == 0):
    co_view_dict[row[session_column_name]].append(row[product_id_column_name])

#build co-view matrix
co_view_matrix = np.zeros((total_products, total_products))
for key, value in co_view_dict.items():
  products_viewed = value
  for i in range(0, len(products_viewed)):
    for j in range(i+1, len(products_viewed)):
      co_view_matrix[products_viewed[i]][products_viewed[j]] = 1
      co_view_matrix[products_viewed[j]][products_viewed[i]] = 1

#print density
density = np.count_nonzero(co_view_matrix) / co_view_matrix.size
print(density)

0.040653661184985354


### **Building co-purchase matrix**

In [ ]:
#co-purchase matrix
co_purchase_dict = {session: [] for session in user_sessions}
#iterate dataframe
for index, row in data_df.iterrows():
  if(row[event_type_column_name] == 1):
    co_purchase_dict[row[session_column_name]].append(row[product_id_column_name])

#build co-view matrix
co_purchase_matrix = np.zeros((total_products, total_products))
for key, value in co_purchase_dict.items():
  products_viewed = value
  for i in range(0, len(products_viewed)):
    for j in range(i+1, len(products_viewed)):
      co_purchase_matrix[products_viewed[i]][products_viewed[j]] = 1
      co_purchase_matrix[products_viewed[j]][products_viewed[i]] = 1

#print density
density_of_co_purchase_matrix = np.count_nonzero(co_purchase_matrix) / co_purchase_matrix.size
print(density_of_co_purchase_matrix)

0.000383425457549098


### **Building Similarity matrix**

In [ ]:
#similarity matrix
category_code_column_name = 'category_code'
brand_column_name = 'brand'

keywords_dict = {}

for index, row in data_df.iterrows():
  product_id = row[product_id_column_name]
  if product_id in keywords_dict:
    continue
  else:
    keywords_dict[product_id] = set()
  brand = row[brand_column_name]
  keywords_dict[product_id].add(brand)
  category = row[category_code_column_name]
  child_categories = category.split('.')
  for child_category in child_categories:
    keywords_dict[product_id].add(child_category)

similar_items_matrix = np.zeros((total_products, total_products))
for i in range(0, total_products):
  for j in range(i+1, total_products):
    keywords_product_i = keywords_dict[i]
    keywords_product_j = keywords_dict[j]
    if(len(keywords_product_i.intersection(keywords_product_j)) > 0):
      similar_items_matrix[i][j] = 1

#print density
density_of_similarity_matrix = np.count_nonzero(similar_items_matrix) / similar_items_matrix.size
print(density_of_similarity_matrix)

0.14297438865821158


### **Increasing the density for co-purchase matrix**

In [ ]:
#Re-building cp matrix
import copy

updated_co_purchase_matrix = copy.deepcopy(co_purchase_matrix)
for i in range(0, total_products):
  for j in range(i+1, total_products):
    if(co_purchase_matrix[i][j] == 1):
      #get all products co-viewed with i
      for k in range(0, total_products):
        if(co_view_matrix[i][k] == 1):
          updated_co_purchase_matrix[i][k] = 1
      #get all products co-viewed with j
      for k in range(0, total_products):
        if(co_view_matrix[j][k] == 1):
          updated_co_purchase_matrix[j][k] = 1
      #get all products similar with i
      for k in range(0, total_products):
        if(similar_items_matrix[i][k] == 1):
          updated_co_purchase_matrix[i][k] = 1
      #get all products similar with j
      for k in range(0, total_products):
        if(similar_items_matrix[j][k] == 1):
          updated_co_purchase_matrix[j][k] = 1

#print density
density_of_updated_co_purchase_matrix = np.count_nonzero(updated_co_purchase_matrix) / updated_co_purchase_matrix.size
print(density_of_updated_co_purchase_matrix)

0.0364919633812844


### **Purchase History**

In [ ]:
user_id_column_name = 'user_id'
purchase_history = {}
for user in range(total_users):
  purchase_history[user] = set()
for index, row in data_df.iterrows():
  if row[event_type_column_name] == 1:
    user_id = row[user_id_column_name]
    purchase_history[user_id].add(row[product_id_column_name])
print(purchase_history)

{0: set(), 1: set(), 2: set(), 3: {601}, 4: set(), 5: set(), 6: set(), 7: set(), 8: set(), 9: set(), 10: set(), 11: set(), 12: set(), 13: set(), 14: {17}, 15: set(), 16: set(), 17: set(), 18: set(), 19: set(), 20: set(), 21: set(), 22: set(), 23: set(), 24: set(), 25: set(), 26: set(), 27: {161, 133, 487, 173, 79, 48, 240, 182, 93}, 28: set(), 29: {528, 57}, 30: {602}, 31: set(), 32: {157}, 33: set(), 34: set(), 35: set(), 36: {305}, 37: set(), 38: set(), 39: set(), 40: set(), 41: set(), 42: set(), 43: {55}, 44: set(), 45: set(), 46: set(), 47: {574}, 48: set(), 49: set(), 50: set(), 51: {581}, 52: set(), 53: {57, 331, 35}, 54: {39}, 55: {48, 97, 614, 30}, 56: set(), 57: set(), 58: set(), 59: set(), 60: set(), 61: set(), 62: {36, 541}, 63: set(), 64: set(), 65: set(), 66: set(), 67: set(), 68: set(), 69: set(), 70: set(), 71: set(), 72: set(), 73: set(), 74: set(), 75: set(), 76: {30}, 77: set(), 78: {35}, 79: {103}, 80: set(), 81: set(), 82: set(), 83: set(), 84: {771}, 85: set(), 86:

### **Popular products**

In [ ]:
popular_products = {}
for product in range(total_products):
  popular_products[product] = 0
for index, row in data_df.iterrows():
  product_id = row[product_id_column_name]
  #view
  if row[event_type_column_name] == 0:
    popular_products[product_id] = popular_products[product_id] + 1
  #purchase
  if row[event_type_column_name] == 1:
    popular_products[product_id] = popular_products[product_id] + 3
  #cart
  if row[event_type_column_name] == 2:
    popular_products[product_id] = popular_products[product_id] + 2
top_k = 12
sorted_dict = sorted(popular_products.items(), key=lambda x: x[1], reverse=True)
top_k_products = [key for key, value in sorted_dict[:top_k]]
print(top_k_products)

[30, 48, 57, 33, 76, 157, 180, 182, 45, 35, 17, 796]


### **Feature Extraction**

In [ ]:
keywords = set()
for index, row in data_df.iterrows():
  brand = row[brand_column_name]
  category = row[category_code_column_name]
  child_categories = category.split('.')
  keywords.add(brand)
  keywords.update(child_categories)

keywords_mapping = {value: index for index, value in enumerate(keywords)}

#Build a matrix of size products * keywords and treat it as input feature matrix
#build co-view matrix
feature_matrix = np.zeros((total_products, len(keywords)))
for key, value in keywords_dict.items():
  product_id = key
  keywords_list = value
  for keyword in keywords_list:
    keyword_id = keywords_mapping[keyword]
    feature_matrix[product_id][keyword_id] = 1

#print density
density_of_feature_matrix = np.count_nonzero(feature_matrix) / feature_matrix.size
print(density_of_feature_matrix)

0.01107210990678019


### **DAEMON IMPLEMENTATION**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

In [ ]:
class GCNLayer(nn.Module):
    r"""
         Implementation of one layer of GCN described in the paper `Semi-Supervised Classification with Graph Convolutional Networks`
    """
    def __init__(self, input_dim, output_dim, activation = F.relu, dropout = None, bias=True):
        super(GCNLayer, self).__init__()
        self.W = nn.Parameter(torch.DoubleTensor(input_dim, output_dim))
        self.activation = activation
        if bias:
            self.b = nn.Parameter(torch.DoubleTensor(output_dim))
        else:
            self.b = None
        if dropout:
            self.dropout = nn.Dropout(p=dropout)
        else:
            self.dropout = None
        self.init_params()

    def init_params(self):
        if self.W is not None:
            init.xavier_uniform_(self.W)
        if self.b is not None:
            init.zeros_(self.b)

    def forward(self, adj, h):
        if self.dropout:
            h = self.dropout(h)
        x = h @ self.W

        x = adj @ x # TODO: not sure if this is right, gotta check dimensions to see if the matrix multiplication is valid
        if self.b is not None:
            x = x + self.b
        if self.activation:
            x = self.activation(x)
        return x

In [ ]:
class CrissCrossGCNLayer(nn.Module):
  r"""
      Implementation of criss cross computations of adj and node representations
  """

  def __init__(self,s_in_feats,s_out_feats,t_in_feats,t_out_feats):
    super(CrissCrossGCNLayer,self).__init__()

    self.layer_s = GCNLayer(s_in_feats,s_out_feats)
    self.layer_t = GCNLayer(t_in_feats,t_out_feats)
  
  def forward(self,h_s,h_t,adj_cv,adj_cp):
    h_s_new = self.layer_s(adj_cv,h_s) + self.layer_t(adj_cp,h_t)
    h_t_new = self.layer_s(adj_cp,h_s) + self.layer_t(adj_cv,h_t)
    
    # update h_s,h_t
    h_s = h_s_new
    h_t = h_t_new

    # normalize with 2-norm
    

    h_s = h_s / torch.norm(h_s,p='fro')
    h_t = h_t / torch.norm(h_t,p='fro')

    return h_s,h_t

In [ ]:
class Recommender(nn.Module):
  r""" Implementation for the DAEMON algorithm
  """
  def __init__(self,in_feats=4,embed_feats=64,out_feats=64,n_layers=3,dropout=0.2):
    super(Recommender,self).__init__()
    self.layers = nn.ModuleList()

    # input layer
    self.layers.append(CrissCrossGCNLayer(s_in_feats=in_feats,s_out_feats=embed_feats,t_in_feats=in_feats,t_out_feats=embed_feats))

    # intermediate layers
    for i in range(n_layers-2):
      self.layers.append(CrissCrossGCNLayer(s_in_feats=embed_feats,s_out_feats=embed_feats,t_in_feats=embed_feats,t_out_feats=embed_feats))

    # output layer
    self.layers.append(CrissCrossGCNLayer(s_in_feats=embed_feats,s_out_feats=out_feats,t_in_feats=embed_feats,t_out_feats=out_feats))

    self.dropout = nn.Dropout(dropout)


  def normalize_adj(self,adj):
    adj.fill_diagonal_(1)
    # normalize adj with A = D^{-1/2} @ A @ D^{-1/2}
    D_norm = torch.diag(torch.pow(adj.sum(1), -0.5)).cuda()
    adj = D_norm @ adj @ D_norm
    return adj
  
  def forward(self,Xu,adj_cp,adj_cv):
    # Xu is for all nodes I suppose (please correct me if im wrong)
    # nodes_in_P is the set of indices where u belongs to P
    h_s = Xu
    h_t = Xu
    # rest two are self explanatory
    adj_cp = self.normalize_adj(adj_cp)
    adj_cv = self.normalize_adj(adj_cv)

    # criss-cross adj matrices
    count = 0
    for layer in self.layers:
      print(count)
      if self.dropout and count!=0:
        # dont apply dropout to the input layer
        h_s = self.dropout(h_s)
        h_t = self.dropout(h_t)
      count += 1
      h_s,h_t = layer(h_s,h_t,adj_cv,adj_cp)
    
    return h_s,h_t

In [ ]:
def compute_loss(theta_s,theta_t,adj_cv,adj_cp,n_k):
  from torch.nn.functional import relu
  
  def dot(x,y):
    # x,y = np.expand_dims(x,1),np.expand_dims(y,1)
    return torch.sum(x * y)
  
  m,n = adj_cv.shape
  term1 = 0
  term_2= 0
  term_3 = 0
  term_4=0
  term5 = 0
  term6 = 0
  z = np.random.randint(low=0,high=theta_s.shape[0]+1,size=n_k) # draw n_k samples from uniform dist
  for u in range(m):
    for z_ in z:
      if u==z_:
        continue
      term_2 += torch.log10(relu(1-dot(theta_s[u],theta_t[z_])))
    for v in range(n):
      if adj_cp[u][v]==1.:
        term1 += torch.log10(relu(dot(theta_s[u],theta_t[v])))
        if adj_cp[v][u]==0.:
          term_3 += torch.log10(relu(dot(theta_s[u],theta_t[v])))
          term_4 += torch.log10(relu(1-dot(theta_s[u],theta_t[v])))
      if adj_cv[u][v]==1.:
        term5 += torch.log10(relu(dot(theta_s[u],theta_s[v])))
        term6 += torch.log10(relu(dot(theta_t[u],theta_t[v])))
  
  return -1*(term1+term_2+term_3+term_4+term5+term6)

In [ ]:
# Optimize and test your recommender model here
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
# initialize all the input data and model below
Xu = torch.from_numpy(feature_matrix).to(device)
adj_cp = torch.from_numpy(co_purchase_matrix).to(device)
adj_cv = torch.from_numpy(co_view_matrix).to(device)
recommender_model = Recommender(in_feats=Xu.shape[1]).to(device) # change embed_feats and out_feats if needed
# initialize optimizer
optimizer = torch.optim.Adam(recommender_model.parameters(), lr = 1e-3, weight_decay = 1e-5)
n_epochs = 30
recommender_model.train() # set to train mode
k = 10
import time

for i in range(n_epochs):
  start_time = time.time()
  theta_s,theta_t = recommender_model(Xu, adj_cp,adj_cv)
  loss = compute_loss(theta_s,theta_t,adj_cv,adj_cp,k) # idk, check paper to see the loss function
  # optmize
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  end_time = time.time()
  time_diff = (start_time-end_time)/60
  print(f'epoch {i}---{time_diff} minutes ---,')
# saving model
torch.save(recommender_model.state_dict(),'./recommender_model.pth')

In [ ]:
################# Test phase ################################
# loading model
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
# initialize all the input data and model below
Xu = torch.from_numpy(feature_matrix).to(device)
adj_cp = torch.from_numpy(co_purchase_matrix).to(device)
adj_cv = torch.from_numpy(co_view_matrix).to(device)
model = Recommender(in_feats=Xu.shape[1]).to(device) # change embed_feats and out_feats if needed
model.load_state_dict(torch.load('/content/drive/My Drive/recommender_model.pth'))
model.eval() # set model to evaluation mode (meaning no gradient computation and param updates)
theta_s,theta_t = model(Xu, adj_cp,adj_cv)

# computing top nk relevancy scores
scores = (theta_s @ theta_t.t()).cpu().detach().numpy()
#number of recommendations
k=12
recommendations_dict = {}
for product in range(len(scores)):
  row = scores[product]
  top_k_indices = np.argpartition(row, -k)[-k:]
  top_k_values = row[top_k_indices]
  # create a list of tuples, where each tuple contains the index and value of a top k element
  pairs = [(i, v) for i, v in zip(top_k_indices, top_k_values)]
  # sort the list of tuples in descending order by value
  pairs.sort(key=lambda x: x[1], reverse=True)
  recommendations_dict[product] = pairs

print(recommendations_dict)
  ################################################################

0
1
2
{0: [(49, 0.0022315216076664443), (101, 0.0022315216076664443), (107, 0.0022315216076664443), (722, 0.0020153148936285113), (115, 0.0018605810543735526), (401, 0.0018098038887492086), (384, 0.00180700441117537), (134, 0.0017728024893492894), (586, 0.001719697968516137), (622, 0.0017187563191625227), (48, 0.0016844255420179627), (57, 0.0016630569145368401)], 1: [(107, 0.0023178765303636753), (101, 0.0023178765303636753), (49, 0.0023178765303636753), (722, 0.0021489890524140592), (115, 0.0019601416420653404), (134, 0.0018703282225421022), (401, 0.00186351591795217), (384, 0.0018551371565815717), (48, 0.0018098709835183424), (622, 0.0017825305775467253), (586, 0.001779614920732042), (57, 0.0017655266722651837)], 2: [(107, 0.0029811394208565634), (49, 0.0029811394208565634), (101, 0.0029811394208565634), (722, 0.0027111073725059835), (115, 0.0024996797739371924), (401, 0.0024003344017581737), (384, 0.002398081055494576), (134, 0.0023806445495068037), (48, 0.0023396552693476166), (586

In [ ]:
recommendations_based_on_purchase = {}
for user in range(total_users):
  k = 12
  recommendations_based_on_purchase[user] = []
  #Get the purchase history
  products_purchased = purchase_history[user]
  if(len(products_purchased) > 0):
    total_list = list()
    #print('products purchased = ', products_purchased)
    for product in products_purchased:
      total_list.extend(recommendations_dict[product])
    sorted_list = sorted(total_list, key=lambda x: x[1], reverse=True)
    my_dict = {}
    for tup in sorted_list:
        if tup[0] not in my_dict:
            my_dict[tup[0]] = tup
    # Step 3: Add the first tuple with a unique key to a new list
    result = []
    for tup in sorted_list:
        if my_dict[tup[0]] == tup:
            result.append(tup)
    size = max(len(result), k)
    #print('sorted recommendations list = ', sorted_list)
    top_k_items = result[:size]
    #print('top k recommendations list = ', top_k_items)
    for item in top_k_items:
      recommendations_based_on_purchase[user].append(item[0])
  else:
    recommendations_based_on_purchase[user].extend(top_k_products)
print(recommendations_based_on_purchase)

{0: [30, 48, 57, 33, 76, 157, 180, 182, 45, 35, 17, 796], 1: [30, 48, 57, 33, 76, 157, 180, 182, 45, 35, 17, 796], 2: [30, 48, 57, 33, 76, 157, 180, 182, 45, 35, 17, 796], 3: [107, 49, 101, 722, 115, 401, 384, 134, 622, 586, 48, 57], 4: [30, 48, 57, 33, 76, 157, 180, 182, 45, 35, 17, 796], 5: [30, 48, 57, 33, 76, 157, 180, 182, 45, 35, 17, 796], 6: [30, 48, 57, 33, 76, 157, 180, 182, 45, 35, 17, 796], 7: [30, 48, 57, 33, 76, 157, 180, 182, 45, 35, 17, 796], 8: [30, 48, 57, 33, 76, 157, 180, 182, 45, 35, 17, 796], 9: [30, 48, 57, 33, 76, 157, 180, 182, 45, 35, 17, 796], 10: [30, 48, 57, 33, 76, 157, 180, 182, 45, 35, 17, 796], 11: [30, 48, 57, 33, 76, 157, 180, 182, 45, 35, 17, 796], 12: [30, 48, 57, 33, 76, 157, 180, 182, 45, 35, 17, 796], 13: [30, 48, 57, 33, 76, 157, 180, 182, 45, 35, 17, 796], 14: [49, 101, 107, 722, 115, 48, 134, 401, 384, 57, 586, 622], 15: [30, 48, 57, 33, 76, 157, 180, 182, 45, 35, 17, 796], 16: [30, 48, 57, 33, 76, 157, 180, 182, 45, 35, 17, 796], 17: [30, 48, 

In [ ]:
print(recommendations_based_on_purchase[3997])

[107, 49, 101, 722, 115, 48, 134, 384, 401, 57, 586, 622]


In [ ]:
print(recommendations_dict[447])

[(49, 0.003525812555831519), (107, 0.003525812555831519), (101, 0.003525812555831519), (722, 0.0032678911619515505), (115, 0.0030037978042916818), (134, 0.002853983966561527), (48, 0.0028467859231808136), (401, 0.002830256742680021), (384, 0.002828624389234374), (57, 0.0027612466167547855), (586, 0.0027540852703039774), (622, 0.002719653874755025)]


In [ ]:
import pickle
pickle.dump(recommendations_based_on_purchase, open('gnn_recommendations_updated', 'wb'))

In [ ]:
#convert purchase_history values from set to list
purchase_history_with_list_values = {
  key: list(value) for key, value in purchase_history.items()
}
print(purchase_history_with_list_values)

{0: [], 1: [], 2: [], 3: [601], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 11: [], 12: [], 13: [], 14: [17], 15: [], 16: [], 17: [], 18: [], 19: [], 20: [], 21: [], 22: [], 23: [], 24: [], 25: [], 26: [], 27: [161, 133, 487, 173, 79, 48, 240, 182, 93], 28: [], 29: [528, 57], 30: [602], 31: [], 32: [157], 33: [], 34: [], 35: [], 36: [305], 37: [], 38: [], 39: [], 40: [], 41: [], 42: [], 43: [55], 44: [], 45: [], 46: [], 47: [574], 48: [], 49: [], 50: [], 51: [581], 52: [], 53: [57, 331, 35], 54: [39], 55: [48, 97, 614, 30], 56: [], 57: [], 58: [], 59: [], 60: [], 61: [], 62: [36, 541], 63: [], 64: [], 65: [], 66: [], 67: [], 68: [], 69: [], 70: [], 71: [], 72: [], 73: [], 74: [], 75: [], 76: [30], 77: [], 78: [35], 79: [103], 80: [], 81: [], 82: [], 83: [], 84: [771], 85: [], 86: [57, 405], 87: [], 88: [], 89: [], 90: [], 91: [], 92: [192, 30], 93: [], 94: [281, 126], 95: [], 96: [], 97: [], 98: [], 99: [], 100: [], 101: [], 102: [], 103: [], 104: [35, 871, 426, 138, 30], 105: []

In [ ]:



import pickle
pickle.dump(purchase_history_with_list_values, open('purchase_history', 'wb'))